# **Return Predictability with Agnostic Fundamental Analysis**
### *Roy Gabriel, Michele Orlandi, Ryan Pate, Michael Smith*
### *MGT6078 Fall 2022*

# 1. **Setup**

In [6]:
import os
import sys
sys.path.append(os.getcwd())
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestRegressor
import gc
import warnings

In [7]:
url = 'https://www.dropbox.com/s/iasfmrfdzafjkyq/student_data.csv?dl=1'

In [8]:
gc.enable()
warnings.filterwarnings('ignore')

In [9]:
# msf = pd.read_csv(url).drop(labels=['PERMNO', 'gvkey', 'COMNAM', 'TICKER', 'SICCD'], axis=1)
# msf = msf.loc[:, msf.columns[1:]].copy()
# gc.collect()
# msf['date'] = pd.to_datetime(msf['date'], format='%Y%m%d')
# msf['nextmonth'] = pd.to_datetime(msf['nextmonth'], format='%Y%m%d')

In [10]:
# msf.head()